In [1]:
import pyslm
from pyslm import hatching as hatching

from libSLM.translators import mtt

from pyslm import geometry

In [2]:
solidPart = pyslm.Part('nut')
solidPart.setGeometry('./stl_artifact/Parameter_quality_nut_1.stl')
solidPart.origin = [5.0, 10.0, 0.0]
solidPart.dropToPlatform()

Geometry information <nut> - [./stl_artifact/Parameter_quality_nut_1.stl]
	 bounds [[ 34.29000092 -39.94499969   0.        ]
 [ 46.99000168 -17.4741745   22.22500038]]
	 extent [12.70000076 22.4708252  22.22500038]
Updating Document Object Geometry Representation


In [3]:
# Create a StripeHatcher object for performing any hatching operations
myHatcher = hatching.StripeHatcher()
myHatcher.stripeWidth = 5.0

# Set the base hatching parameters which are generated within Hatcher
myHatcher.hatchAngle = 10 # [°]
myHatcher.volumeOffsetHatch = 0.08 # [mm]
myHatcher.spotCompensation = 0.06 # [mm]
myHatcher.numInnerContours = 2
myHatcher.numOuterContours = 1

In [4]:
model = geometry.Model()
model.mid = 1
model.name = "Nut"

In [5]:
header = geometry.Header()
header.filename = "MTT Layerfile"
header.version = (1,2)
header.zUnit = 1000

In [6]:
BuildStyle = geometry.BuildStyle()
BuildStyle.bid = 1

# Set the laser parameters for the Contour Build Style
BuildStyle.laserPower = 200  # W
BuildStyle.laserSpeed = 500  # mm/s - Note this is used on some systems but should be set
BuildStyle.laserFocus = 0.0  # mm - (Optional) Some new systems can modify the focus position real-time.

# The point exposure parameters are specified for some systems (typically Q-Switch Pulse Lasers)
# Note: the laser speed v = pointDistance / pointExposureTime
BuildStyle.pointDistance = 50  # μm - Distance between exposure points
BuildStyle.pointExposureTime = 80  # ms - Exposure time

In [7]:
model.buildStyles.append(BuildStyle)

In [8]:
import numpy as np

layer_thickness = 0.1

layer_list = []
for i in np.arange(0.0, solidPart.boundingBox[5], layer_thickness):
    myHatcher.hatchAngle += 66.7
    
    geomSlice = solidPart.getVectorSlice(i)
    layer = myHatcher.hatch(geomSlice)
    
    for geo in layer.geometry:
        geo.mid = model.mid
        geo.bid = BuildStyle.bid
    
    # The layer height is set in integer increment of microns to ensure no rounding error during manufacturing
    layer.z = int(i*layer_thickness*1000)
    layer_list.append(layer)
    

Updating Document Object Geometry Representation
bounding box bbox [ 39.67   -24.1328  51.61   -13.579 ]
bounding box bbox [ 39.67   -23.7512  51.61   -13.9606]
bounding box bbox [ 39.67   -23.6139  51.61   -14.098 ]
bounding box bbox [ 39.67   -23.5197  51.61   -14.1921]
bounding box bbox [ 39.67   -23.4441  51.61   -14.2677]
bounding box bbox [ 39.67   -23.4027  51.61   -14.3092]
bounding box bbox [ 39.67   -23.3646  51.61   -14.3473]
bounding box bbox [ 39.67   -23.3514  51.61   -14.3604]
bounding box bbox [ 39.67   -23.3399  51.61   -14.372 ]
bounding box bbox [ 39.67   -23.353   51.61   -14.3588]
bounding box bbox [ 39.67   -23.3664  51.61   -14.3454]
bounding box bbox [ 39.67   -23.4079  51.61   -14.304 ]
bounding box bbox [ 39.67   -23.4508  51.61   -14.2593]
bounding box bbox [ 39.67   -23.5085  51.61   -14.1825]
bounding box bbox [ 39.67   -23.5663  51.61   -14.0953]
bounding box bbox [ 39.67   -23.624   51.61   -13.9953]
bounding box bbox [ 39.67   -23.6817  51.61   -13.8953]

bounding box bbox [ 39.67   -15.1822  51.61    -7.8541]
bounding box bbox [ 39.67   -29.4437  51.61   -22.2369]
bounding box bbox [ 39.67   -15.1953  51.61    -7.8541]
bounding box bbox [ 39.67   -29.386   51.61   -22.2237]
bounding box bbox [ 39.67   -15.2085  51.61    -7.8541]
bounding box bbox [ 39.67   -29.3282  51.61   -22.2106]
bounding box bbox [ 39.67   -15.2217  51.61    -7.8541]
bounding box bbox [ 39.67   -29.2705  51.61   -22.1974]
bounding box bbox [ 39.67   -15.2348  51.61    -7.8541]
bounding box bbox [ 39.67   -29.2128  51.61   -22.1842]
bounding box bbox [ 39.67   -15.248   51.61    -7.8541]
bounding box bbox [ 39.67   -29.155   51.61   -22.1711]
bounding box bbox [ 39.67   -15.2612  51.61    -7.8541]
bounding box bbox [ 39.67   -29.0973  51.61   -22.1579]
bounding box bbox [ 39.67   -15.2993  51.61    -7.8541]
bounding box bbox [ 39.67   -29.0396  51.61   -22.1198]
bounding box bbox [ 39.67   -15.3407  51.61    -7.8541]
bounding box bbox [ 39.67   -28.9818  51.61   -2

In [9]:
export_mtt = mtt.Writer()
export_mtt.setFilePath('./output/mtt_file.mtt')
export_mtt.write(header, [model], layer_list)